In [1]:
import numpy as np
import scipy.linalg as sln
import matplotlib.pyplot as plt
import warnings
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

Points
<br># -> Doc(Main)
<br>## -> My Notes
<br>### -> Missing, Corrections to do

In [2]:
def mv(A,x,transp_flag=0):
    if np.issubdtype(A.dtype, np.number):
        ##Check this def in matlab as well
        if np.any(A):
            if transp_flag == 0:
                x = A @ x
            else:
                x = A.T @ x

In [3]:
def unv(j, n):
    e = np.zeros(n)
    e[j] = 1
    return e

In [4]:
def element(A, i=None, j=None, n=None):

    # Handling default arguments
    if i is None:
        i = slice(None)  # Selecting all rows
    if j is None:
        j = slice(None)  # Selecting all columns

    if isinstance(A, np.ndarray):  # Matrix
        if isinstance(i, list) and isinstance(j, list):
            e = A[np.ix_(i, j)]
        else:
            e = A[i, j]
    else:  # Function
        if j is None:
            raise ValueError("j has to be nonempty when A is a function")
        e = mv(A, unv(j, n), 0)
        if i is not None:
            e = e[i]

    return e

In [5]:
A = np.random.randn(4, 4)
print(A)
element(A,[1,2],[2,3])

[[ 2.760  0.660 -0.343 -0.957]
 [ 1.608 -1.501  2.058  0.222]
 [ 0.386  0.223  0.651 -0.909]
 [ 1.222 -1.157  0.658  0.665]]


array([[ 2.058,  0.222],
       [ 0.651, -0.909]])

In [6]:
###Get rid of function aspect
def element(A, i=None, j=None, n=None):
    # Handling default arguments
    if i is None:
        i = slice(None)  # Selecting all rows
    if j is None:
        j = slice(None)  # Selecting all columns

    if isinstance(A, np.ndarray):  # Matrix
        if min(A.shape) > 1:  # Check if A is not a vector
            if isinstance(i, list) and isinstance(j, list):
                e = A[np.ix_(i, j)]
            else:
                e = A[i, j]
        else:
            e = A[i] if isinstance(i, list) else A[i]
    else:  # Function
        if j is None:
            raise ValueError("j has to be nonempty when A is a function")
        e = mv(A, unv(j, n), 0)
        if i is not None:
            e = e[i]

    return e

def unv(j, n):
    e = np.zeros(n)
    e[j] = 1
    return e

In [7]:
A = np.random.randn(4, 4)
print(A)
element(A,[1,2],[2,3])


[[-1.263  0.213  1.118 -0.172]
 [ 0.572 -0.596 -1.139  0.851]
 [-1.346  0.692  1.447  1.071]
 [-2.296  0.305  1.161  0.192]]


array([[-1.139,  0.851],
       [ 1.447,  1.071]])

In [8]:
def krylov_ata(A, v1=None, k=10, full=1, reortho=2):
    if v1 is None:
        v1 = np.random.randn(A.shape[1])

    if not np.issubdtype(A.dtype, np.number):
        raise ValueError("Matrix A should be numeric")

    alpha = np.zeros(k)
    beta = np.zeros(k if full else k-1)

    if reortho:
        V = np.zeros((len(v1), k + 1))
        V[:, 0] = v1 / np.linalg.norm(v1)
        U = np.zeros((A.shape[0], k))
    else:
        v = v1 / np.linalg.norm(v1)

    for j in range(k):
        if reortho:
            r = mv(A, V[:, j], 0)
            if j == 0 and reortho == 2:
                U = np.zeros((len(r), k))
        else:
            r = mv(A, v, 0)

        if j > 0:
            if reortho == 2:
                r -= beta[j-1] * U[:, j-1]
                r -= U[:, :j] @ (U[:, :j].T @ r)
            else:
                r -= beta[j-1] * u

        alpha[j] = np.linalg.norm(r)
        if alpha[j] == 0:
            break

        if reortho == 2:
            U[:, j] = r / alpha[j]
            r = mv(A, U[:, j], 1)
        else:
            u = r / alpha[j]
            r = mv(A, u, 1)

        if reortho:
            r -= alpha[j] * V[:, j]
            r -= V[:, :j+1] @ (V[:, :j+1].T @ r)
        else:
            r -= alpha[j] * v

        if j < k - 1 or full:
            beta[j] = np.linalg.norm(r)
            if beta[j] == 0:
                break

            if reortho:
                V[:, j+1] = r / beta[j]
            else:
                v = r / beta[j]

    if not reortho:
        V = v
    if reortho < 2:
        U = u

    if full:
        return V[:, :k+1], U, alpha, beta
    else:
        return V[:, :k], U, alpha, beta[:k-1]


In [9]:
A = np.array([[1,2,3,4],[3,5,2,8],[5,3,45,6],[2,1,6,3]])
print(A)
V,U,alpha,beta = krylov_ata(A)
print(V)
print(U)
print(alpha)
print(beta)

[[ 1  2  3  4]
 [ 3  5  2  8]
 [ 5  3 45  6]
 [ 2  1  6  3]]


TypeError: object of type 'NoneType' has no len()

In [10]:
def krylov_ata_expand(A, V, U, c, k = 10):
    m = V.shape[1]
    V = np.concatenate((V, np.zeros((V.shape[0],k))), axis = 1)
    U = np.concatenate((U, np.zeros((U.shape[0],k))), axis = 1)
    alpha = np.zeros(k)
    beta = np.zeros(k)

    for j in range(m - 1, k + m - 1):
        if j == m - 1:
            ###Check this section
            print((U[:, :j - 2]).shape)
            print(c.shape)
            print((mv(A, V[:, j - 1], 0)).shape)
            #r = mv(A, V[:, j - 1], 0) - U[:, :j - 2] @ c
            r = mv(A, V[:, j - 1], 0) - U[:, :j - 2] @ c
        else:
            r = mv(A, V[:, j - 1],0) - beta[j - m - 1] * U[:, j - 2]

        r = r - U[:, :j - 2] @ (U[:, :j - 2].T @ r)
        alpha[j - m] = np.linalg.norm(r)
        U[:, j - 1] = r / alpha[j - m]
        r = mv(A.T, U[:, j - 1]) - alpha[j - m] * V[:, j - 1]
        r = r - V[:, :j] @ (V[:, :j].T @ r)
        beta[j - m] = np.linalg.norm(r)
        V[:, j] = r / beta[j - m]

    return V, U, alpha, beta

In [11]:
def krylov_schur_svd(A, **kwargs):
    #Unpacking kwargs to collect the parameters
    nr = kwargs.get('nr',None)
    v1 = kwargs.get('v1',None)
    tol = kwargs.get('tol',None)
    absrel = kwargs.get('absrel',None)
    mindim = kwargs.get('mindim',None)
    maxdim = kwargs.get('maxdim',None)
    maxit = kwargs.get('maxit',None)
    target = kwargs.get('target',None)
    info = kwargs.get('info',None)

    #Checking is A is numeric
    ###Need to fix when A is a function
    if not (np.issubdtype(A.dtype, np.number)):
        return ("matrix entered is not numeric in nature")

    #Giving a value for the unpacked parameters in kwargs
    if nr is None:
        nr = 1
    if v1 is None:
        v1 = np.random.rand(A.shape[1])
    if tol is None:
        tol = 1e-6
    if absrel is None:
        absrel = 'rel'
    if mindim is None:
        mindim = 10
    if maxdim is None:
        maxdim = 20
    if maxit is None:
        maxit = 1000
    if target is None:
        target = np.inf
    if info is None:
        info = 0

    #Reassigning maxdim and mindim to shorter variables
    m1 = mindim
    m2 = maxdim

    #Fixing some of the parameters
    if m1 < nr:
        m1 = nr
    if m2 < m1:
        m2 = 2 * m1
    if (absrel == 'rel') and (np.issubdtype(A.dtype, np.number)):
        ##Multiplying the tol with the 1-norm of A
        tol = tol * np.linalg.norm(A, 1)

    B = np.zeros((m2,m2 + 1))
    V, U, alpha, beta = krylov_ata(A, v1, m1)
    B[:m1 + 1, :m1 +1] = np.diag(np.append(alpha, [0])) + np.diag(beta, 1)
    hist = np.zeros(maxit - 1)
    
    for k in range(maxit):
        V, U, alpha, beta = krylov_ata_expand(A, V, U, B[:m1, m1], m2 - m1)
        ##Subtracting to match the 'm2-m1' index in MATLAB
        B[m1:, m1: m2 + 1] = np.diag(alpha) + np.diag(beta[:m2 - m1 -1], 1) 
        B[m2 - 1, m2] = beta[m2 - m1 - 1]
        X, sigma, Y = np.linalg.svd(B[:m2, :m2])
    
        ####No need to attempt the target == 0 section####
            
        
        #Restarts of Lancos algorithm
        result = element(A, list(np.arange(1, 3)), list(np.arange(2)))
        ###V = np.concatenate(((element(V[:, :m2] @ Y, 0:n, 0:m1), V[:, m2])))
        ###U = element(U[:, :m2] @ X, 0:m, 0:m1)
        ###e = element(B[:, m2].T @ X, 0:m1)
        V = np.concatenate(((element(V[:, :m2] @ Y, list(np.arange(0,n)), list(np.arange(0,m1))), V[:, m2])))
        U = element(U[:, :m2] @ X, list(np.arange(0,m)), list(np.arange(0,m1)))
        e = element(B[:, m2].T @ X, list(np.arange(0,m1)))
        ###Check the axis of concatenation
        B[:m1, :m1 + 1] = np.concatenate((np.diag(sigma[:m1]), e.T))
    
        err = np.linalg.norm(e[:nr])
        hist[k] = err
    
        if (info > 1):
            print(f'{k:4d}  {err:6.2e}')
            print(f'{sigma[:min(3,nr)]:6.2e}')
    
        if (err < tol):
            sigma = sigma[:nr]
            V = V[:, :nr]
            U = U[:, :nr]
            mvs = np.arange(1, k + 1) * (m2 - m1) + m1
            print("Found after {k} iterations with residual = {err:6.2e}".format(k, err))
            return sigma, V, U, hist, mvs
    if info:
            print("Quit after max {k} iterations with residual = {err:6.2e}".format(k, err))
            sigma = sigma[:m1]
            V = V[:, :m1]
    return sigma, V, U, hist, mvs
            

In [12]:
sigma, V, U, hist, mvs = krylov_schur_svd(A, **opts)

NameError: name 'opts' is not defined

In [13]:
A = np.array([[1,2,3,4],[3,5,2,8],[5,3,45,6],[2,1,6,3]])
A

array([[ 1,  2,  3,  4],
       [ 3,  5,  2,  8],
       [ 5,  3, 45,  6],
       [ 2,  1,  6,  3]])

In [14]:
opts = {
        'nr': 2,
        'v1': np.random.randn(A.shape[1]),
        'tol': 1e-6,
        'absrel': 'rel',
        'mindim': 10,
        'maxdim': 20,
        'maxit': 1000,
        'target': np.inf,
        'info': 2
    }


In [15]:
import numpy as np

def mv(A, v, transp_flag):
    if transp_flag == 0:
        return A @ v
    else:
        return A.T @ v

def krylov_ata_expand(A, V, U, c, k=10):
    m = V.shape[1]
    V = np.concatenate((V, np.zeros((V.shape[0], k))), axis=1)
    U = np.concatenate((U, np.zeros((U.shape[0], k))), axis=1)
    alpha = np.zeros(k)
    beta = np.zeros(k)

    for j in range(m - 1, k + m):
        if j == m - 1:
            ###Check difference with imatlab
            r = mv(A, V[:, j-1], 0) - U[:, :j-1] @ c[:j-1]
        else:
            r = mv(A, V[:, j-1], 0) - beta[j-m-1] * U[:, j-2]

        r = r - U[:, :j-1] @ (U[:, :j-1].T @ r)
        alpha[j-m] = np.linalg.norm(r)
        if alpha[j-m] == 0:
            break
        U[:, j-1] = r / alpha[j-m]
        r = mv(A.T, U[:, j-1], 1) - alpha[j-m] * V[:, j-1]
        r = r - V[:, :j] @ (V[:, :j].T @ r)
        beta[j-m] = np.linalg.norm(r)
        if beta[j-m] == 0:
            break
        V[:, j] = r / beta[j-m]

    return V, U, alpha, beta

# Set the random seed for reproducibility
np.random.seed(0)

# Define the input matrices and vectors
A = np.random.randn(4, 4)
V = np.random.randn(4, 3)
U = np.random.randn(4, 3)
c = np.random.randn(3)
k = 5

# Call the krylov_ata_expand function
V_exp, U_exp, alpha, beta = krylov_ata_expand(A, V, U, c, k)

# Display the results
print("V_exp:")
print(V_exp)
print("U_exp:")
print(U_exp)
print("alpha:")
print(alpha)
print("beta:")
print(beta)


V_exp:
[[ 1.494 -0.205  0.034  0.127  0.143  0.151  0.154  0.155]
 [-0.854 -2.553 -0.974 -0.966 -0.963 -0.961 -0.960 -0.959]
 [ 0.864 -0.742 -0.153 -0.133 -0.117 -0.112 -0.109 -0.108]
 [-1.454  0.046 -0.162 -0.180 -0.198 -0.205 -0.208 -0.209]]
U_exp:
[[ 1.533 -0.744 -0.762 -0.795 -0.793 -0.793 -0.793  0.000]
 [ 0.378  0.472 -0.185 -0.107 -0.113 -0.113 -0.113  0.000]
 [-0.348 -0.358  0.171  0.109  0.114  0.114  0.114  0.000]
 [ 1.202 -0.309 -0.597 -0.587 -0.588 -0.588 -0.588  0.000]]
alpha:
[ 85.325  2888.391  138149.794  8719817.875  700701388.744]
beta:
[ 650.053  25271.069  1348364.590  93840234.885  8241617450.114]


In [16]:
# Fixed input matrices and vectors
A = np.array([
    [1, 2, 3, 4],
    [4, 5, 6, 7],
    [7, 8, 9, 10],
    [10, 11, 12, 13]
])
V = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])
U = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])
c = np.array([1, 2, 3])
k = 2

# Call the krylov_ata_expand function
V_exp, U_exp, alpha, beta = krylov_ata_expand(A, V, U, c, k)

# Display the results
print("V_exp:")
print(V_exp)
print("U_exp:")
print(U_exp)
print("alpha:")
print(alpha)
print("beta:")
print(beta)

V_exp:
[[ 1.000  2.000  0.111  0.111  0.111]
 [ 4.000  5.000  0.328  0.328  0.328]
 [ 7.000  8.000  0.546  0.546  0.546]
 [ 10.000  11.000  0.763  0.763  0.763]]
U_exp:
[[ 1.000 -0.077 -0.078 -0.078  0.000]
 [ 4.000 -0.310 -0.310 -0.310  0.000]
 [ 7.000 -0.543 -0.543 -0.543  0.000]
 [ 10.000 -0.776 -0.776 -0.776  0.000]]
alpha:
[ 62808872265.078  3980876294657805.000]
beta:
[ 23837582699001.832  1514825833565455616.000]


In [29]:
def krylov_schur_svd(A, **kwargs):
    nr = kwargs.get('nr', 1)
    v1 = kwargs.get('v1', None)
    tol = kwargs.get('tol', 1e-6)
    absrel = kwargs.get('absrel', 'rel')
    mindim = kwargs.get('mindim', 10)
    maxdim = kwargs.get('maxdim', 20)
    maxit = kwargs.get('maxit', 1000)
    target = kwargs.get('target', np.inf)
    info = kwargs.get('info', 0)

    if v1 is None:
        v1 = np.random.rand(A.shape[1])
    
    if mindim < nr:
        mindim = nr
    if maxdim < 2 * mindim:
        maxdim = 2 * mindim
    
    if absrel == 'rel' and np.issubdtype(A.dtype, np.number):
        tol = tol * np.linalg.norm(A, 1)

    B = np.zeros((maxdim, maxdim + 1))
    V, U, alpha, beta = krylov_ata(A, v1, mindim)
    B[:mindim + 1, :mindim + 1] = np.diag(np.append(alpha, [0])) + np.diag(beta, 1)
    hist = np.zeros(maxit)

    for k in range(maxit):
        V, U, alpha, beta = krylov_ata_expand(A, V, U, B[:mindim, mindim], maxdim - mindim)
        B[mindim: maxdim, mindim: maxdim] = np.diag(alpha) + np.diag(beta[:maxdim - mindim - 1], 1)
        B[maxdim - 1, maxdim] = beta[maxdim - mindim - 1]
        X, sigma, Y = np.linalg.svd(B[:maxdim, :maxdim])
        
        # Restart of Lanczos algorithm
        V = np.concatenate((element(V[:, :maxdim] @ Y, list(range(V.shape[0])), list(range(mindim))), V[:, maxdim:maxdim + 1]), axis=1)
        U = element(U[:, :maxdim] @ X, list(range(U.shape[0])), list(range(mindim)))
        c = B[:, maxdim]
        e = (c @ X)[:mindim]
        B[:mindim, :mindim + 1] = np.concatenate((np.diag(sigma[:mindim]), e.reshape(-1, 1)), axis=1)
        
        err = np.linalg.norm(e[:nr])
        hist[k] = err
        
        if info > 1:
            print(f'{k:4d}  {err:6.2e}')
            print(sigma[:min(3,nr)])
        
        if err < tol:
            sigma = sigma[:nr]
            V = V[:, :nr]
            U = U[:, :nr]
            mvs = np.arange(1, k + 1) * (maxdim - mindim) + mindim
            print(f"Found after {k} iterations with residual = {err:6.2e}")
            return sigma, V, U, hist[:k+1], mvs
    
    if info:
        print(f"Quit after max {k} iterations with residual = {err:6.2e}")
    sigma = sigma[:mindim]
    V = V[:, :mindim]
    return sigma, V, U, hist, mvs
    
A = np.random.rand(50, 50)
sigma, V, U, hist, mvs = krylov_schur_svd(A, nr=3, tol=1e-4, maxit=100, info=2)

   0  2.44e-03
[ 25.110  3.947  3.561]
Found after 0 iterations with residual = 2.44e-03
